## Title

## Introduction

- Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal
- Clearly state the question you will try to answer with your project
- Identify and describe the dataset that will be used to answer the question

## Preliminary exploratory data analysis
- Demonstrate that the dataset can be read from the web into R 
- Clean and wrangle your data into a tidy format
- **Using only training data**, summarize the data in at least one table (this is exploratory data analysis). An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data. 
- **Using only training data**, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.

## Methods
- Explain how you will conduct either your data analysis and which variables/columns you will use. <ins>Note - you do not need to use all variables/columns that exist in the raw data set. In fact, that's often not a good idea. For each variable think: is this a useful variable for prediction?</ins>
- Describe at least one way that you will visualize the results


To conduct our data analysis, we will use knn to try to classify whether a patient has heart disease. In our dataset, the variable we are looking to predict is represented by the num column (where 0 means healthy and 1-4 means sick). 

First, we will **select** the variables we wish to use as predictors. We will use age, sex, trestbps (resting blood pressure), cholesteral, and max heart rate as our predictors and an exploratory analysis of these predictors was conducted above. For ease of use, we are using only the variables that are giving numeric values. We will put all of these variables into a **recipe** and both **scale** and **center** all predictors. 

Next, we will create a **model specification** that will use straight line distance and tune for the best k value. 

After, we will perform a **5-fold cross-validation** on the training set and create a **workflow analysis** that includes both our recipe and model specification. We will try all integer k-values from 2 - 10. This is to determine what the most effective/accurate k-value to use is. 

When that is done, we will plot the results and visually see how many neighbours to use.

Finally, we will create our model by making a new **model specification** that will use straight line distance and the value of k that we determined previously. We will add both this new specification and the recipe we made before to a new **workflow**. We will use this model to **predict** the values in our testing set. We will also check our model's **accuracy**, **precision**, and **recall** using **metrics** and the **confusion matrix**.

One of the ways that we will visualize these results is by using scatterplots to compare two variables against the class we are predicting (represented by colour) and then plot the test observations. By doing this, we will be able to visually see whether our model is working well; we can look and see whether our model seems to have correctly predicted what class of heart disease our patient has based on the colours of the k-nearest neighbours surrounding our point.

## Expected outcomes and significance
- What do you expect to find?
- What impact could such findings have?
- What future questions could this lead to?
